This notebook compares analytical and numerical evaluations of alchemical derivatives of a rotated and translated CO molecule up to the third order in Hartree-Fock theory (HFT).  
The results are summerized in the last section.

In [1]:
from pyscf import gto,scf
import numpy as np
import pyscf
from scipy.spatial.transform import Rotation

pyscf.__version__

'2.2.1'

In [2]:
from aqa.FcMole import FcM_like

Rotating and translating CO with the coordinate [[0, 0, 0], [0, 0, 2.1]].

In [3]:
mol_coord = np.array([[0, 0, 0], [0, 0, 2.1]])
rotvec = np.array([2 * np.pi/1.1, np.pi/2, np.pi/3])
rot = Rotation.from_rotvec(rotvec)
mol_coord = rot.apply(mol_coord)
# print(mol_coord)
# print(np.linalg.norm(mol_coord[1]))
mol_coord[0] += [13.4, 21.9, 3.8]
mol_coord[1] += [13.4, 21.9, 3.8]
print(mol_coord)
print(np.linalg.norm(mol_coord[1] - mol_coord[0]))

[[13.4        21.9         3.8       ]
 [13.26749486 22.4300496   5.82768089]]
2.1


In [4]:
# target_mol = "C 0 0 0; O 0 0 2.1"
target_mol = "C %f %f %f; O %f %f %f" % (mol_coord[0,0], mol_coord[0,1],
                                         mol_coord[0,2], mol_coord[1,0],
                                         mol_coord[1,1], mol_coord[1,2])
# dft_functional = "pbe0"  # "lda,vwn"
basis_set = "def2-TZVP"
print(target_mol)

C 13.400000 21.900000 3.800000; O 13.267495 22.430050 5.827681


### Analytical evaluation

In [5]:
%load_ext autoreload
from aqa.AP_class import APDFT_perturbator as AP

In [6]:
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
mf_nn=scf.RHF(mol_NN)
# mf_nn.xc = dft_functional
mf_nn.scf()
# mf_nn.conv_tol = 1e-12
# mf_nn.grids.level = 6
# mf_nn.nuc_grad_method().kernel()
ap_nn=AP(mf_nn,sites=[0,1],flag_response_property=True)

converged SCF energy = -112.78696838997


In [7]:
analytical_1st_alchemical_derivative = ap_nn.build_gradient()
analytical_2nd_alchemical_derivative = ap_nn.build_hessian()
analytical_3rd_alchemical_derivative = ap_nn.build_cubic()

Inefficient alchemical force is not implemented in the Hartree-Fock theory

Inefficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [8]:
# inefficient_analytical_alchemical_force = np.zeros((2, 2, 3))
# inefficient_analytical_alchemical_force[0] = ap_nn.build_inefficient_alchemical_force(0)
# inefficient_analytical_alchemical_force[1] = ap_nn.build_inefficient_alchemical_force(1)

Efficient alchemical force in HFT which calculates the response matrix for nuclear coordinates

In [9]:
efficient_analytical_alchemical_force = np.zeros((2, 2, 3))
efficient_analytical_alchemical_force[0] = ap_nn.build_alchemical_force(0)
efficient_analytical_alchemical_force[1] = ap_nn.build_alchemical_force(1)

Electric dipole derivatives $\partial ^2 E/ \partial Z_I \partial \mathbf{F}$

In [10]:
elec_electric_dipole_gradient_z = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='Z')
elec_electric_dipole_gradient_f = ap_nn.build_elec_electric_dipole_gradient(perturb_electron_density='F')

Electric polarizability derivatives $\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F}$

In [11]:
elec_electric_pol_gradient = ap_nn.build_electric_polarizability_gradient()

### Numerical evaluation

In [12]:
%autoreload 2
mol_NN=gto.M(atom=target_mol,unit="Bohr",basis=basis_set)
fc_param = 0.001
fcs1 = [fc_param, 0.0]
fcs2 = [-fc_param, 0.0]
fcs3 = [0.0, fc_param]
fcs4 = [0.0, -fc_param]

##### Efficient numerical alchemical force

In [13]:
fmol1=FcM_like(mol_NN,fcs=fcs1)
fmol2=FcM_like(mol_NN,fcs=fcs2)
fmol3=FcM_like(mol_NN,fcs=fcs3)
fmol4=FcM_like(mol_NN,fcs=fcs4)
mf=scf.RHF(mol_NN)
mf1=scf.RHF(fmol1)
mf2=scf.RHF(fmol2)
mf3=scf.RHF(fmol3)
mf4=scf.RHF(fmol4)
# mf.xc = dft_functional
# mf1.xc = dft_functional
# mf2.xc = dft_functional
# mf3.xc = dft_functional
# mf4.xc = dft_functional
# mf.conv_tol = 1e-12
# mf1.conv_tol = 1e-12
# mf2.conv_tol = 1e-12
# mf3.conv_tol = 1e-12
# mf4.conv_tol = 1e-12
# mf.grids.level = 6
# mf1.grids.level = 6
# mf2.grids.level = 6
# mf3.grids.level = 6
# mf4.grids.level = 6
# Without dm0=mf.init_guess_by_1e(), some SCFs do not converge.
e=mf.scf(dm0=mf.init_guess_by_1e())
e1=mf1.scf(dm0=mf1.init_guess_by_1e())
e2=mf2.scf(dm0=mf2.init_guess_by_1e())
e3=mf3.scf(dm0=mf3.init_guess_by_1e())
e4=mf4.scf(dm0=mf4.init_guess_by_1e())

converged SCF energy = -112.786968389975
converged SCF energy = -112.801621811359


/home/takafumi/program/pyscf/pyscf/gto/mole.py:1213: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -112.772317850945
converged SCF energy = -112.809217058244
converged SCF energy = -112.764723302573


In [14]:
nuc_grad = mf.nuc_grad_method().kernel()
nuc_grad1 = mf1.nuc_grad_method().kernel()
nuc_grad2 = mf2.nuc_grad_method().kernel()
nuc_grad3 = mf3.nuc_grad_method().kernel()
nuc_grad4 = mf4.nuc_grad_method().kernel()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0014685757    -0.0058746353    -0.0224731371
1 O    -0.0014685757     0.0058746353     0.0224731371
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0014854890    -0.0059422923    -0.0227319558
1 O    -0.0014854890     0.0059422923     0.0227319558
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0014516618    -0.0058069760    -0.0222143098
1 O    -0.0014516618     0.0058069760     0.0222143098
----------------------------------------------
--------------- RHF gradients ---------------
         x                y                z
0 C     0.0014714985    -0.0058863270    -0.0225178634
1 O    -0.0014714985     0.0058863270     0.0225178634
----------------------------------------------
--------

In [15]:
efficient_numerical_alchemical_force = np.zeros((2, 2, 3))
efficient_numerical_alchemical_force[0] = (nuc_grad1 - nuc_grad2) / (2 * fc_param)
efficient_numerical_alchemical_force[1] = (nuc_grad3 - nuc_grad4) / (2 * fc_param)

#### Numerical alchemical derivatives

In [16]:
numerical_1st_alchemical_derivative = np.zeros((2))
numerical_1st_alchemical_derivative[0] = (e1 - e2) / (2 * fc_param)
numerical_1st_alchemical_derivative[1] = (e3 - e4) / (2 * fc_param)

#### Numerical alchemical hardness

In [17]:
ap_nn=AP(mf,sites=[0,1])
ap_nn1=AP(mf1,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn2=AP(mf2,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn3=AP(mf3,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
ap_nn4=AP(mf4,sites=[0,1],flag_response_property=True,charges_for_center=[6,8])
an = ap_nn.build_gradient()
an1 = ap_nn1.build_gradient()
an2 = ap_nn2.build_gradient()
an3 = ap_nn3.build_gradient()
an4 = ap_nn4.build_gradient()
ann = ap_nn.build_hessian()
ann1 = ap_nn1.build_hessian()
ann2 = ap_nn2.build_hessian()
ann3 = ap_nn3.build_hessian()
ann4 = ap_nn4.build_hessian()
dipole1 = ap_nn1.ref_elec_electric_dipole_moment * -1
dipole2 = ap_nn2.ref_elec_electric_dipole_moment * -1
dipole3 = ap_nn3.ref_elec_electric_dipole_moment * -1
dipole4 = ap_nn4.ref_elec_electric_dipole_moment * -1
pol1 = ap_nn1.ref_elec_electric_polarizability * -1
pol2 = ap_nn2.ref_elec_electric_polarizability * -1
pol3 = ap_nn3.ref_elec_electric_polarizability * -1
pol4 = ap_nn4.ref_elec_electric_polarizability * -1

In [18]:
numerical_2nd_alchemical_derivative = np.zeros((2, 2))
numerical_2nd_alchemical_derivative[0] = (an1 - an2) / (2 * fc_param)
numerical_2nd_alchemical_derivative[1] = (an3 - an4) / (2 * fc_param)

#### Numerical alchemical hardness derivatives

In [19]:
numerical_3rd_alchemical_derivative = np.zeros((2, 2, 2))
numerical_3rd_alchemical_derivative[0, :, :] = (ann1 - ann2) / (2 * fc_param)
numerical_3rd_alchemical_derivative[1, :, :] = (ann3 - ann4) / (2 * fc_param)

#### Inefficient numerical alchemical force

In [20]:
# mol_NN1=gto.M(atom="C 0 0 0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN2=gto.M(atom="C 0 0 -0.001; O 0 0 2.1",unit="Bohr",basis=basis_set)
# mol_NN3=gto.M(atom="C 0 0 0; O 0 0 2.101",unit="Bohr",basis=basis_set)
# mol_NN4=gto.M(atom="C 0 0 0; O 0 0 2.099",unit="Bohr",basis=basis_set)
# mf_nn1=scf.RKS(mol_NN1)
# mf_nn2=scf.RKS(mol_NN2)
# mf_nn3=scf.RKS(mol_NN3)
# mf_nn4=scf.RKS(mol_NN4)
# mf_nn1.xc = dft_functional
# mf_nn2.xc = dft_functional
# mf_nn3.xc = dft_functional
# mf_nn4.xc = dft_functional
# mf_nn1.scf()
# mf_nn2.scf()
# mf_nn3.scf()
# mf_nn4.scf()
# ap_nn1=AP(mf_nn1,sites=[0,1])
# ap_nn2=AP(mf_nn2,sites=[0,1])
# ap_nn3=AP(mf_nn3,sites=[0,1])
# ap_nn4=AP(mf_nn4,sites=[0,1])
# ad1 = ap_nn1.build_gradient()
# ad2 = ap_nn2.build_gradient()
# ad3 = ap_nn3.build_gradient()
# ad4 = ap_nn4.build_gradient()

In [21]:
# inefficient_numerical_alchemical_force = np.zeros((2, 2))  # only z axis components
# inefficient_numerical_alchemical_force[:, 0] = (ad1 - ad2) / (2 * fc_param)
# inefficient_numerical_alchemical_force[:, 1] = (ad3 - ad4) / (2 * fc_param)

#### Numerical alchemical electric dipole moment

In [22]:
numer_elec_electric_dipole_gradient = np.zeros((2, 3))
numer_elec_electric_dipole_gradient[0] = (dipole1 - dipole2) / (2 * fc_param)
numer_elec_electric_dipole_gradient[1] = (dipole3 - dipole4) / (2 * fc_param)

#### Numerical alchemical electric polarizability

In [23]:
numer_elec_electric_pol_gradient = np.zeros((2, 3, 3))
numer_elec_electric_pol_gradient[0] = (pol1 - pol2) / (2 * fc_param)
numer_elec_electric_pol_gradient[1] = (pol3 - pol4) / (2 * fc_param)

## Comparison of analytical and numerical evaluations

$ \partial E/\partial Z_i $

In [24]:
print(analytical_1st_alchemical_derivative)

[-14.65197964 -22.24687957]


$ (\partial E/\partial Z_i)_\mathrm{analytical} $ - $ (\partial E/\partial Z_i)_\mathrm{numerical} $

In [25]:
print(analytical_1st_alchemical_derivative - numerical_1st_alchemical_derivative)

[ 5.64142313e-07 -1.73739281e-06]


$\partial^2E/\partial Z_i\partial Z_j$

In [26]:
print(analytical_2nd_alchemical_derivative)

[[-2.88235455  0.46095804]
 [ 0.46095804 -3.58086849]]


$(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial Z_j)_\mathrm{numerical}$

In [27]:
print(analytical_2nd_alchemical_derivative - numerical_2nd_alchemical_derivative)

[[ 4.81800898e-07 -4.12086565e-07]
 [-1.32539035e-10 -3.49815361e-07]]


$\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [28]:
print(analytical_3rd_alchemical_derivative)

[[[-0.13175057  0.13763009]
  [ 0.13763009  0.08208885]]

 [[ 0.13763009  0.08208885]
  [ 0.08208885 -0.16691397]]]


$(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{analytical}$ - $(\partial^3E/\partial Z_i\partial Z_j\partial Z_k)_\mathrm{numerical}$

In [29]:
print(analytical_3rd_alchemical_derivative - numerical_3rd_alchemical_derivative)

[[[ 2.59155497e-07 -1.11104627e-08]
  [-1.11104627e-08 -1.51667706e-07]]

 [[-6.20814956e-08 -1.97341449e-07]
  [-1.97341449e-07  3.51899334e-07]]]


$\partial^2E/\partial Z_i\partial R_j$

In [30]:
print(efficient_analytical_alchemical_force)

[[[ 0.01691359 -0.06765818 -0.25882316]
  [-0.01691359  0.06765818  0.25882316]]

 [[ 0.00292903 -0.01171678 -0.04482197]
  [-0.00292903  0.01171678  0.04482197]]]


$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [31]:
print(efficient_analytical_alchemical_force - efficient_numerical_alchemical_force)

[[[ 1.22229649e-08 -4.89025080e-08 -1.87071318e-07]
  [-1.22262420e-08  4.89049495e-08  1.87075767e-07]]

 [[-9.82495568e-09  3.93059434e-08  1.50354680e-07]
  [ 9.82462461e-09 -3.93072803e-08 -1.50344006e-07]]]


$\partial^2E/\partial Z_i\partial R_j$

In [32]:
# print(inefficient_analytical_alchemical_force)

$(\partial^2E/\partial Z_i\partial R_j)_\mathrm{analytical}$ - $(\partial^2E/\partial Z_i\partial R_j)_\mathrm{numerical}$

In [33]:
# print(inefficient_analytical_alchemical_force[:, :, 2] - inefficient_numerical_alchemical_force)

$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$

In [34]:
print(elec_electric_dipole_gradient_f)

[[ 0.03722054 -0.14889059 -0.56957385]
 [-0.07985331  0.31943133  1.22196931]]


In [35]:
print(elec_electric_dipole_gradient_z)

[[ 0.03722054 -0.14889059 -0.56957384]
 [-0.07985332  0.31943134  1.22196936]]


$(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{analytical}$ - $(\partial ^2 E/ \partial Z_I \partial \mathbf{F})_\mathrm{numerical}$

In [36]:
print(elec_electric_dipole_gradient_f - numer_elec_electric_dipole_gradient)

[[-7.40476746e-08  2.96210546e-07  1.13315131e-06]
 [ 1.88597174e-07 -7.54424807e-07 -2.88601841e-06]]


In [37]:
print(elec_electric_dipole_gradient_z - numer_elec_electric_dipole_gradient)

[[-7.44155013e-08  2.97681936e-07  1.13878003e-06]
 [ 1.85514535e-07 -7.42093551e-07 -2.83884578e-06]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$

In [38]:
print(elec_electric_pol_gradient)

[[[ 7.31312445  0.02460162  0.09411231]
  [ 0.02460162  7.22086247 -0.37647053]
  [ 0.09411231 -0.37647053  5.87910445]]

 [[ 3.73999108 -0.06555006 -0.25075864]
  [-0.06555006  3.98581958  1.00309135]
  [-0.25075864  1.00309135  7.5608826 ]]]


$(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{analytical}$ - $(\partial ^3 E/ \partial Z_I \partial ^2 \mathbf{F})_\mathrm{numerical}$

In [39]:
print(elec_electric_pol_gradient - numer_elec_electric_pol_gradient)

[[[-8.46778392e-06  8.16927225e-08  3.12457294e-07]
  [ 8.16927225e-08 -8.77407713e-06 -1.24982330e-06]
  [ 3.12457294e-07 -1.24982330e-06 -1.32284959e-05]]

 [[-5.68828756e-06 -2.83968217e-08 -1.08617245e-07]
  [-2.83968217e-08 -5.58180505e-06  4.34502021e-07]
  [-1.08617245e-07  4.34502021e-07 -4.03329032e-06]]]
